In [1]:
import sys

sys.path.append('../../qick/qick_lib/')
sys.path.append('soft/')

from qick import *
from qick_training import *

import numpy as np
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt

In [2]:
# Load bitstream with custom overlay
soc = QickSoc('./qick_training.bit')
soccfg = soc

print(soccfg)

resetting clocks: 245.76 491.52

QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 349.997, RF reference 245.760

	6 signal generator channels:
	0:	axis_sg_int4_v1 - tProc output 0, envelope memory 4096 samples
		DAC tile 2, blk 0, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	1:	axis_sg_int4_v1 - tProc output 1, envelope memory 4096 samples
		DAC tile 2, blk 1, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	2:	axis_sg_int4_v1 - tProc output 2, envelope memory 4096 samples
		DAC tile 2, blk 2, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	3:	axis_sg_int4_v1 - tProc output 3, envelope memory 4096 samples
		DAC tile 2, blk 3, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	4:	axis_signal_gen_v6 - tProc output 4, envelope memory 65536 samples
		DAC tile 3, blk 0, 32-bit DDS, fabric=307.200 MHz, f_dds=4915.200 MHz
	5:	axis_sg_mux4_v1 - tProc output 6, envelope memory 0 samples
		DAC tile 0, blk 0, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz



In [83]:
class SineWaveTest(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        gen_ch = cfg["gen_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["gen_ch"], nqz=1)
        
        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"], gen_ch=cfg['gen_ch'])
        gain = cfg["pulse_gain"]
        
        self.default_pulse_registers(ch=gen_ch, 
                                     style="const", 
                                     freq=freq, 
                                     gain=gain, 
                                     length=self.us2cycles(cfg["pulse_length"],gen_ch=cfg['gen_ch']))
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.trigger(pins=[0],t=40)
             
        self.set_pulse_registers(ch=self.cfg["gen_ch"], 
                                 phase=soccfg.deg2reg(self.cfg['phase'],self.cfg['gen_ch']),
                                 phrst=self.cfg['phrst'], 
                                 mode="oneshot")
        self.pulse(ch=self.cfg["gen_ch"],t=self.us2cycles(self.cfg['pulse_start'], gen_ch=self.cfg['gen_ch']))
       
        # Period.
        self.synci(self.us2cycles(self.cfg['period']))
       
config={"gen_ch"      : 1,
        "reps"        : 20000000,
        "style"       : 'const',
        "pulse_length": 10,
        "pulse_start" : 3,
        "pulse_gain"  : 30000, # [DAC units]
        "pulse_freq"  : 10, # [MHz]
        "period"      : 20,
        "phase"       : 0,
        "phrst"       : 1,
       }

prog = SineWaveTest(soccfg, config)
prog.config_all(soccfg)
soc.tproc.start()

In [72]:
class MultiOutTest(AveragerProgram):
    def initialize(self):
        # Set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"][0], nqz=1)
        self.declare_gen(ch=self.cfg["gen_ch"][1], nqz=1)
        
        # First channel.
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][0], 
                                     style  = self.cfg['style'], 
                                     freq   = self.freq2reg(self.cfg['pulse_freq'][0],gen_ch=self.cfg['gen_ch'][0]),
                                     gain   = self.cfg['pulse_gain'][0], 
                                     length = self.us2cycles(self.cfg['pulse_length'][0],gen_ch=self.cfg['gen_ch'][0]))
        
        # Second channel.
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][1], 
                                     style  = self.cfg['style'], 
                                     freq   = self.freq2reg(self.cfg['pulse_freq'][1],gen_ch=self.cfg['gen_ch'][1]),
                                     gain   = self.cfg['pulse_gain'][1], 
                                     length = self.us2cycles(self.cfg['pulse_length'][1],gen_ch=self.cfg['gen_ch'][1]))        
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # Registers.
        self.set_pulse_registers(ch=self.cfg['gen_ch'][0], 
                                 phase=soccfg.deg2reg(self.cfg['phase'][0],self.cfg['gen_ch'][0]),
                                 phrst=self.cfg['phrst'][0], 
                                 mode="oneshot")
        self.set_pulse_registers(ch=self.cfg['gen_ch'][1], 
                                 phase=soccfg.deg2reg(self.cfg['phase'][1],self.cfg['gen_ch'][1]),
                                 phrst=self.cfg['phrst'][1], 
                                 mode="oneshot")
        # Trigger.
        self.trigger(pins=[0],t=40)

        # Pulses.
        self.pulse(ch     = self.cfg['gen_ch'][0],
                   t      = self.us2cycles(self.cfg['pulse_start'][0], 
                   gen_ch = self.cfg['gen_ch'][0]))
        
        self.pulse(ch     = self.cfg['gen_ch'][1],
                   t      = self.us2cycles(self.cfg['pulse_start'][1], 
                   gen_ch = self.cfg['gen_ch'][1]))
        
        # Period.
        self.synci(self.us2cycles(self.cfg['period']))
       
config={"gen_ch"      : [1,3],
        "reps"        : 70000000,
        "style"       : 'const',
        "pulse_length": [12,12],
        "pulse_start" : [3,3],
        "pulse_gain"  : [30000,30000], # [DAC units]
        "pulse_freq"  : [3,3], # [MHz]
        "period"      : 20,
        "phase"       : [0,180],
        "phrst"       : [1,1]
       }

prog = MultiOutTest(soccfg, config)
prog.config_all(soccfg)
soc.tproc.start()

In [ ]:
print(prog)